# mai maiの収録曲をyotubeで検索してjupyterに埋め込むツール


## seleniumのセットアップ

In [ ]:
from selenium import webdriver
options = webdriver.ChromeOptions()
options.add_argument('--headless')
driver = webdriver.Chrome(options=options)

## maimaiの楽曲リストを取得

In [ ]:
driver.get('https://maimai.sega.jp/song/niconico/')

## 楽曲をリスト化し、チェックボックスを付与

In [ ]:
import ipywidgets as widgets
songs = {el.text:widgets.Checkbox(
    value=False,
    description=el.text,
    disabled=False
) for el in driver.find_elements_by_class_name('titleText')}

_ = [display(w) for w in songs.values()]

## チェックされた曲を検索対象とする

In [ ]:
songs_search = [song for song,w in songs.items() if w.value]
print('search for',songs_search)

## youtubeのapiで検索し、videoIdのリストを返却する関数
DEVELOPER_KEYは各自用意したもので置き換え
### 参考
- apiの[pythonクライアント](https://developers.google.com/api-client-library/python/?hl=ja)
- 検索パラメータの[公式リファレンス](https://developers.google.com/youtube/v3/?hl=ja)


In [ ]:
from apiclient.discovery import build
from apiclient.errors import HttpError

DEVELOPER_KEY = "REPLACE_ME"
YOUTUBE_API_SERVICE_NAME = "youtube"
YOUTUBE_API_VERSION = "v3"

def youtube_search(term, count=1):
  youtube = build(YOUTUBE_API_SERVICE_NAME, YOUTUBE_API_VERSION,
    developerKey=DEVELOPER_KEY)

  search_response = youtube.search().list(
    q=term,
    part="id",
    type="video",
    maxResults=count
  ).execute()

  return [item['id']['videoId'] for item in search_response.get("items", [])]

## notebookに埋め込むHTMLを作成する関数
`_repr_html_`関数はプライベートメソッドなのでアレかもです。コード読んでたら見つけました。  
あるいは(横に並べたいので)`<iframe></iframe><iframe></iframe>...`って書いてく感じになるかと。

In [ ]:
from IPython.display import YouTubeVideo,HTML
# 内部で呼ばれてる_repr_html_でhtmlを取り出して加工
def html_youtube(title,video_ids):
    return HTML(''.join([
    f'<h2>{title}</h2>'
    ,*[YouTubeVideo(f'{id}')._repr_html_() for id in video_ids]
    ]
    ))

In [ ]:
w_count = widgets.Dropdown(
    options=['1', '2', '3'],
    value='2',
    description='検索数:',
    disabled=False,
)
display(w_count)

## youtube埋め込み

In [ ]:
songs_youtube = [{'title':song,'video_ids':youtube_search(song,w_count.value)} for song in songs_search]
_ = [display(html_youtube(**song)) for song in songs_youtube]